In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Tutorial

Import library:

In [ ]:
import ratioimage.ratio_images as ri
import ratioimage.plot_results as rp

ModuleNotFoundError: No module named 'py_clone_detective'

Load in images and segmentations (label images):

In [ ]:
C0_imgs = ri.create_img_dict_from_folder(load_pattern="data/PercevalHR_data/imgs/*C0.tiff")
C1_imgs = ri.create_img_dict_from_folder(load_pattern="data/PercevalHR_data/imgs/*C1.tiff")
lab_imgs = re.create_img_dict_from_folder(
    load_pattern="data/PercevalHR_data/segs/*.tif", label=True
)

NameError: name 'ri' is not defined

In [ ]:
df = ratiometric_measure_region_props_to_tidy_df(C1_imgs, C0_imgs, lab_imgs)
df = split_img_key_col_to_sample_id_and_replicates(
    df, sample_id_colname="genotype", split_regex=r"g"
)

df = sample_id_to_categories(
    df, sample_id_colname="genotype", old_to_new_sample_ids={"a1": "ctrl", "a2": "mut"}
)

df.head()

ModuleNotFoundError: No module named 'py_clone_detective'